In [15]:
import pandas as pd
import numpy as np
from dotenv import load_dotenv
import os
from neo4j import GraphDatabase
from langchain_community.graphs import Neo4jGraph

In [3]:
load_dotenv()

True

In [20]:
NEO4J_URL = os.getenv('NEO4J_URL')
NEO4J_USERNAME = os.getenv('NEO4J_USERNAME')
NEO4J_PASSWORD = os.getenv('NEO4J_PASSWORD')
NEO4J_DATABASE = os.getenv('NEO4J_DATABASE') or 'neo4j'
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')



In [8]:
crew_df = pd.read_csv('clean_data/crew_clean.csv')
crew_df.head()

,tconst,directors,writers
0,tt0000012,"nm0525908,nm0525910",\N
1,tt0000417,nm0617588,"nm0617588,nm0894523,nm0920229"
2,tt0000439,nm0692105,"nm1145809,nm0692105"
3,tt0006864,nm0000428,"nm0048512,nm0115218,nm0000428,nm0002616,nm0640..."
4,tt0009968,nm0000428,"nm0121885,nm0000428"


In [9]:
movie_ratings_df = pd.read_csv('clean_data/movie_ratings_clean.csv')
movie_ratings_df.head()

,tconst,averageRating,numVotes
0,tt0000012,7.4,12824
1,tt0000417,8.1,55524
2,tt0000439,7.3,20924
3,tt0006864,7.7,16683
4,tt0009968,7.2,11035


In [10]:
movie_title_df = pd.read_csv('clean_data/movie_title_clean.csv')
movie_title_df.head()

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
0,tt0000012,short,The Arrival of a Train,L'arrivée d'un train à La Ciotat,0,1896.0,\N,1.0,"Documentary,Short"
1,tt0000417,short,A Trip to the Moon,Le voyage dans la lune,0,1902.0,\N,13.0,"Action,Adventure,Comedy"
2,tt0000439,short,The Great Train Robbery,The Great Train Robbery,0,1903.0,\N,11.0,"Action,Adventure,Crime"
3,tt0006864,movie,Intolerance,Intolerance: Love's Struggle Throughout the Ages,0,1916.0,\N,163.0,"Drama,History"
4,tt0009968,movie,Broken Blossoms,Broken Blossoms or The Yellow Man and the Girl,0,1919.0,\N,90.0,"Drama,Romance"


In [11]:
names_df = pd.read_csv('clean_data/names_clean.csv')
names_df.head()

,nconst,primaryName,birthYear,deathYear,primaryProfession,knownForTitles
0,nm0000001,Fred Astaire,1899,1987,"actor,miscellaneous,producer","tt0072308,tt0050419,tt0053137,tt0027125"
1,nm0000002,Lauren Bacall,1924,2014,"actress,soundtrack,archive_footage","tt0037382,tt0075213,tt0117057,tt0038355"
2,nm0000003,Brigitte Bardot,1934,\N,"actress,music_department,producer","tt0057345,tt0049189,tt0056404,tt0054452"
3,nm0000004,John Belushi,1949,1982,"actor,writer,music_department","tt0072562,tt0077975,tt0080455,tt0078723"
4,nm0000005,Ingmar Bergman,1918,2007,"writer,director,actor","tt0050986,tt0083922,tt0050976,tt0069467"


In [12]:
principals_df = pd.read_csv('clean_data/principals_clean.csv')
principals_df.head()

,tconst,ordering,nconst,category,job,characters
0,tt0000012,1,nm2880396,self,\N,"[""Self""]"
1,tt0000012,2,nm9735580,self,\N,"[""Self""]"
2,tt0000012,3,nm0525900,self,\N,"[""Self""]"
3,tt0000012,4,nm9735581,self,\N,"[""Self""]"
4,tt0000012,5,nm9735579,self,\N,"[""Self""]"


In [13]:
movie_desc_df = pd.read_csv('clean_data/movie_description_clean_embeddings.csv')
movie_desc_df.head()

,Unnamed: 0,tconst,primaryTitle,About,Embeddings
0,0,tt0000012,The Arrival of a Train,"""The Arrival of a Train"" is a 1896 silent shor...","[-0.6185780167579651, 0.3374672532081604, 0.87..."
1,1,tt0000417,A Trip to the Moon,"""A Trip to the Moon"" (1902) is a classic silen...","[-0.09322287887334824, 0.7676082849502563, 0.3..."
2,2,tt0000439,The Great Train Robbery,"""The Great Train Robbery"" is a 1978 British cr...","[0.700336217880249, 0.25397372245788574, 0.077..."
3,3,tt0006864,Intolerance,"""Intolerance"" (1916) is a silent epic directed...","[-0.0787254050374031, 0.4562748074531555, 0.61..."
4,4,tt0009968,Broken Blossoms,"""Broken Blossoms"" is a 1919 silent film direct...","[-1.0412063598632812, -0.3301662802696228, 0.7..."


In [131]:
embedded_titles_df = pd.merge(movie_desc_df[['tconst','Embeddings','About']], movie_title_df[['tconst','titleType','primaryTitle','originalTitle','isAdult','startYear','endYear','runtimeMinutes','genres']], on = 'tconst')
embedded_titles_df.head()

,tconst,Embeddings,About,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
0,tt0000012,"[-0.6185780167579651, 0.3374672532081604, 0.87...","""The Arrival of a Train"" is a 1896 silent shor...",short,The Arrival of a Train,L'arrivée d'un train à La Ciotat,0,1896.0,\N,1.0,"Documentary,Short"
1,tt0000417,"[-0.09322287887334824, 0.7676082849502563, 0.3...","""A Trip to the Moon"" (1902) is a classic silen...",short,A Trip to the Moon,Le voyage dans la lune,0,1902.0,\N,13.0,"Action,Adventure,Comedy"
2,tt0000439,"[0.700336217880249, 0.25397372245788574, 0.077...","""The Great Train Robbery"" is a 1978 British cr...",short,The Great Train Robbery,The Great Train Robbery,0,1903.0,\N,11.0,"Action,Adventure,Crime"
3,tt0006864,"[-0.0787254050374031, 0.4562748074531555, 0.61...","""Intolerance"" (1916) is a silent epic directed...",movie,Intolerance,Intolerance: Love's Struggle Throughout the Ages,0,1916.0,\N,163.0,"Drama,History"
4,tt0009968,"[-1.0412063598632812, -0.3301662802696228, 0.7...","""Broken Blossoms"" is a 1919 silent film direct...",movie,Broken Blossoms,Broken Blossoms or The Yellow Man and the Girl,0,1919.0,\N,90.0,"Drama,Romance"


In [56]:
neo4j_driver = GraphDatabase.driver(
    uri=NEO4J_URL, 
    auth = (NEO4J_USERNAME,NEO4J_PASSWORD), 
    database=NEO4J_DATABASE,
    connection_timeout = 50*60
)

In [ ]:
from neo4j import GraphDatabase

def update_movie_ratings(driver, df, batch_size=100):
    query = """
    UNWIND $ratings AS rating
    MATCH (m:Movie {tconst: rating.tconst})
    SET m.averageRating = rating.averageRating, m.numVotes = rating.numVotes
    RETURN count(m) AS updatedCount
    """

    with driver.session() as session:
        total_rows = len(df)
        total_batches = (total_rows + batch_size - 1) // batch_size

        for i in range(total_batches):
            start_index = i * batch_size
            end_index = min(start_index + batch_size, total_rows)
            batch = df.iloc[start_index:end_index]
            ratings = batch.to_dict(orient='records')
            
            # Define a transaction function
            def transaction_function(tx):
                result = tx.run(query, ratings=ratings)
                return result.single()[0]  # Fetch and return the result within the transaction

            # Execute the transaction
            updated_count = session.write_transaction(transaction_function)
            print(f"Batch {i+1} updated: {updated_count} movie(s).")

# update_movie_ratings(neo4j_driver, movie_ratings_df, batch_size=100)


In [ ]:
def add_movies_with_embeddings(df, batch_size=100):
    query = """
    UNWIND $movies as movie
    MERGE (m:Movie {tconst: movie.tconst})
    ON CREATE SET m.titleType = movie.titleType, m.primaryTitle = movie.primaryTitle,
                  m.originalTitle = movie.originalTitle, m.isAdult = movie.isAdult,
                  m.startYear = movie.startYear, m.endYear = movie.endYear,
                  m.runtimeMinutes = movie.runtimeMinutes, m.genres = movie.genres,
                  m.embedding = movie.embedding
    """
    with neo4j_driver.session() as session:
        # Iterate over batches and execute transactions
        for start in range(0, len(df), batch_size):
            end = start + batch_size
            batch = df.iloc[start:end]
            movies = batch.to_dict(orient='records')
            
            # Execute the query within a transaction
            def transaction_function(tx):
                result = tx.run(query, movies=movies)
                return result.consume()  # Consume the result within the transaction scope
            
            summary = session.execute_write(transaction_function)
            print(f"Batch {start//batch_size + 1} updated with embeddings: {summary.counters}")



In [67]:
def add_persons(df, batch_size=100):
    # Create a unique constraint on the Person nodes for the nconst property
    constraint_query = "CREATE CONSTRAINT IF NOT EXISTS FOR (p:Person) REQUIRE p.nconst IS UNIQUE"
    
    with neo4j_driver.session() as session:
        session.execute_write(lambda tx: tx.run(constraint_query))
        print("Unique constraint on nconst set for Person nodes.")

        # Process data in batches
        total_batches = (len(df) + batch_size - 1) // batch_size  # Calculate the number of batches
        for i in range(total_batches):
            start_index = i * batch_size
            end_index = start_index + batch_size
            batch = df.iloc[start_index:end_index]

            # Convert batch DataFrame to a list of dictionaries for batch processing
            persons = batch.to_dict(orient='records')

            # Define the query to merge Person nodes into the database
            person_query = """
            UNWIND $persons as person
            MERGE (p:Person {nconst: person.nconst})
            ON CREATE SET p.primaryName = person.primaryName, p.birthYear = person.birthYear,
                          p.deathYear = person.deathYear, p.primaryProfession = person.primaryProfession
            """
            try:
                session.execute_write(lambda tx: tx.run(person_query, {'persons': persons}))
                print(f"Batch {i+1}/{total_batches} of Person nodes added or updated.")
            except Exception as e:
                print(f"Error processing batch {i+1}: {str(e)}")
                # Optionally, you could add retry logic or other error handling here


In [138]:
def set_unique_constraint():
    query = "CREATE CONSTRAINT movieId_unique IF NOT EXISTS FOR (m:Movie) REQUIRE m.tconst IS UNIQUE"
    try:
        neo4j_driver.execute_query(query)
        print("Constraint created successfully.")
    except Exception as e:
        print(f"Failed to create constraint: {str(e)}")

set_unique_constraint()


Constraint created successfully.


In [96]:
def create_vector_index():
    index_query = """
    CREATE VECTOR INDEX moviePlots IF NOT EXISTS
    FOR (m:Movie)
    ON m.embedding
    OPTIONS {
        indexConfig: {
            `vector.dimensions`: 1024,
            `vector.similarity_function`: 'cosine'
        }
    }
    """
    try:
        neo4j_driver.execute_query(index_query)
        print("Vector index created successfully.")
    except Exception as e:
        print(f"Failed to create vector index: {str(e)}")

In [88]:
def add_crew_relationships(df, batch_size=1):
    with neo4j_driver.session() as session:
        total_batches = (len(df) + batch_size - 1) // batch_size  # Calculate the number of batches

        for i in range(total_batches):
            start_index = i * batch_size
            end_index = start_index + batch_size
            batch = df.iloc[start_index:end_index]

            # Prepare director data for the batch
            directors_data = [
                {'tconst': row['tconst'], 'nconst': director}
                for _, row in batch.iterrows() for director in row['directors'].split(',') if row['directors'] != '\\N'
            ]

            # Prepare writer data for the batch
            writers_data = [
                {'tconst': row['tconst'], 'nconst': writer}
                for _, row in batch.iterrows() for writer in row['writers'].split(',') if row['writers'] != '\\N'
            ]

            # Create Directed By relationships
            directors_query = """
            UNWIND $data as row
            MATCH (m:Movie {tconst: row.tconst}), (p:Person {nconst: row.nconst})
            MERGE (p)-[:DIRECTED_BY]->(m)
            """
            if directors_data:  # Only run if there is data to process
                session.execute_write(lambda tx: tx.run(directors_query, {'data': directors_data}))

            # Create Written By relationships
            writers_query = """
            UNWIND $data as row
            MATCH (m:Movie {tconst: row.tconst}), (p:Person {nconst: row.nconst})
            MERGE (p)-[:WRITTEN_BY]->(m)
            """
            if writers_data:  # Only run if there is data to process
                session.execute_write(lambda tx: tx.run(writers_query, {'data': writers_data}))

            print(f"Batch {i+1}/{total_batches} of crew relationships added.")


In [124]:
def add_principal_relationships(df, batch_size=100):
    with neo4j_driver.session() as session:
        total_batches = (len(df) + batch_size - 1) // batch_size  # Calculate the number of batches

        for i in range(total_batches):
            start_index = i * batch_size
            end_index = start_index + batch_size
            batch = df.iloc[start_index:end_index]

            # Convert batch DataFrame to a list of dictionaries for batch processing
            principal_data = batch.to_dict(orient='records')

            # Define the query to merge Principal relationships into the database
            principals_query = """
            UNWIND $data as row
            MATCH (m:Movie {tconst: row.tconst}), (p:Person {nconst: row.nconst})
            MERGE (p)-[:ACTED_IN {ordering: row.ordering, job: row.job, characters: row.characters}]->(m)
            """
            if principal_data:  # Only run if there is data to process
                session.execute_write(lambda tx: tx.run(principals_query, {'data': principal_data}))

            print(f"Batch {i+1}/{total_batches} of principal relationships added.")


In [ ]:
def update_movie_embeddings(df, batch_size=100):
    # Define the query to update movie embeddings
    query = """
    UNWIND $movies AS row
    MATCH (m:Movie {tconst: row.tconst})
    CALL db.create.setNodeVectorProperty(m, 'embedding', apoc.convert.fromJsonList(row.Embeddings))
    RETURN count(*)
    """

    # Initialize session
    with neo4j_driver.session() as session:
        total_rows = len(df)
        total_batches = (total_rows + batch_size - 1) // batch_size  # Calculate the number of batches

        for i in range(total_batches):
            start_index = i * batch_size
            end_index = min(start_index + batch_size, total_rows)  # Ensure we do not exceed the dataframe length
            batch = df.iloc[start_index:end_index]
            movies = batch.to_dict(orient='records')

            try:
                # Execute the query within a transaction
                result = session.execute_write(lambda tx: tx.run(query, {'movies': movies}))
                updated_count = result.single()[0]  # Get the count of updated nodes
                print(f"Batch {i+1} updated with embeddings: {updated_count} movies updated.")
            except Exception as e:
                print(f"Error updating embeddings in batch {i+1}: {str(e)}")

# Example usage
# Assuming df is your DataFrame loaded with movie data including 'tconst' and 'Embeddings' columns
update_movie_embeddings(embedded_titles_df, batch_size=100)


In [144]:
create_vector_index()

Vector index created successfully.


In [ ]:
def update_movie_genres_in_batches(df, batch_size=100):
    query = """
    UNWIND $movies AS row
    MATCH (m:Movie {tconst: row.tconst})
    SET m.genres = split(m.genres, ',')
    RETURN count(*)
    """

    with neo4j_driver.session() as session:
        total_rows = len(df)
        total_batches = (total_rows + batch_size - 1) // batch_size  # Calculate the number of batches

        for i in range(total_batches):
            start_index = i * batch_size
            end_index = min(start_index + batch_size, total_rows)  # Ensure we do not exceed the dataframe length
            batch = df.iloc[start_index:end_index]
            movies = batch.to_dict(orient='records')

            try:
                # Execute the query within a transaction
                result = session.write_transaction(lambda tx: tx.run(query, movies=movies))
                updated_count = result.single()[0]  # Get the count of updated nodes
                print(f"Batch {i+1} updated: {updated_count} movies' genres converted to list.")
            except Exception as e:
                print(f"Error updating genres in batch {i+1}: {str(e)}")

# Example usage
# Assuming df is your DataFrame loaded with movie data including 'tconst' and 'genres' columns
update_movie_genres_in_batches(embedded_titles_df, batch_size=100)


In [ ]:
add_persons(names_df)

In [ ]:
add_crew_relationships(crew_df)

In [ ]:
add_principal_relationships(principals_df)

In [141]:
def update_or_add_movies_with_embeddings(df, batch_size=100):
    query = """
    UNWIND $movies as movie
    MERGE (m:Movie {tconst: movie.tconst})
    ON CREATE SET m.titleType = movie.titleType, m.primaryTitle = movie.primaryTitle,
                  m.originalTitle = movie.originalTitle, m.isAdult = movie.isAdult,
                  m.startYear = movie.startYear, m.endYear = movie.endYear,
                  m.runtimeMinutes = movie.runtimeMinutes, m.genres = split(movie.genres, ','),
                  m.About = movie.About
    ON MATCH SET m.genres = split(movie.genres, ','), m.About = movie.About
    """
    with neo4j_driver.session() as session:
        # Iterate over batches and execute transactions
        for start in range(0, len(df), batch_size):
            end = start + batch_size
            batch = df.iloc[start:end]
            movies = batch.to_dict(orient='records')
            
            # Execute the query within a transaction
            def transaction_function(tx):
                result = tx.run(query, movies=movies)
                return result.consume()  # Consume the result within the transaction scope
            
            summary = session.execute_write(transaction_function)
            print(f"Batch {start//batch_size + 1} updated with embeddings and about: {summary.counters}")

# Example usage:
# Assuming df is your DataFrame loaded with movie data including 'tconst' and 'genres', 'About', and other relevant fields
update_or_add_movies_with_embeddings(embedded_titles_df, batch_size=100)


Batch 1 updated with embeddings and about: {'_contains_updates': True, 'properties_set': 300}
Batch 2 updated with embeddings and about: {'_contains_updates': True, 'properties_set': 300}
Batch 3 updated with embeddings and about: {'_contains_updates': True, 'properties_set': 300}
Batch 4 updated with embeddings and about: {'_contains_updates': True, 'properties_set': 300}
Batch 5 updated with embeddings and about: {'_contains_updates': True, 'properties_set': 300}
Batch 6 updated with embeddings and about: {'_contains_updates': True, 'properties_set': 300}
Batch 7 updated with embeddings and about: {'_contains_updates': True, 'properties_set': 300}
Batch 8 updated with embeddings and about: {'_contains_updates': True, 'properties_set': 300}
Batch 9 updated with embeddings and about: {'_contains_updates': True, 'properties_set': 300}
Batch 10 updated with embeddings and about: {'_contains_updates': True, 'properties_set': 300}
Batch 11 updated with embeddings and about: {'_contains_upd